In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
data_gold_usd = pd.read_csv("XAU_USD.csv")
data_gold_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1291 non-null   object 
 1   Price     1291 non-null   object 
 2   Open      1291 non-null   object 
 3   High      1291 non-null   object 
 4   Low       1291 non-null   object 
 5   Vol.      0 non-null      float64
 6   Change %  1291 non-null   object 
dtypes: float64(1), object(6)
memory usage: 70.7+ KB


In [6]:
df = data_gold_usd.drop(['Open','High','Low','Vol.','Change %'], axis=1)

In [7]:
df.head(10)

,Date,Price
0,12/15/2022,"1,776.42"
1,12/14/2022,"1,807.74"
2,12/13/2022,"1,810.29"
3,12/12/2022,"1,781.15"
4,12/09/2022,"1,796.63"
5,12/08/2022,"1,789.23"
6,12/07/2022,"1,786.22"
7,12/06/2022,"1,770.91"
8,12/05/2022,"1,768.39"
9,12/02/2022,"1,797.82"


In [9]:
df['Price'] = df['Price'].apply(lambda x: float(x.split()[0].replace(',', '')))
df

,Date,Price
0,12/15/2022,1776.42
1,12/14/2022,1807.74
2,12/13/2022,1810.29
3,12/12/2022,1781.15
4,12/09/2022,1796.63
...,...,...
1286,01/05/2018,1320.21
1287,01/04/2018,1322.97
1288,01/03/2018,1312.84
1289,01/02/2018,1318.14


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1291 non-null   object 
 1   Price   1291 non-null   float64
dtypes: float64(1), object(1)
memory usage: 20.3+ KB


In [12]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			 break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = df['Price']
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1776.42 1807.74 1810.29 1781.15 1796.63] 1789.23
[1807.74 1810.29 1781.15 1796.63 1789.23] 1786.22
[1810.29 1781.15 1796.63 1789.23 1786.22] 1770.91
[1781.15 1796.63 1789.23 1786.22 1770.91] 1768.39
[1796.63 1789.23 1786.22 1770.91 1768.39] 1797.82
[1789.23 1786.22 1770.91 1768.39 1797.82] 1802.89
[1786.22 1770.91 1768.39 1797.82 1802.89] 1768.45
[1770.91 1768.39 1797.82 1802.89 1768.45] 1749.73
[1768.39 1797.82 1802.89 1768.45 1749.73] 1740.75
[1797.82 1802.89 1768.45 1749.73 1740.75] 1756.14
[1802.89 1768.45 1749.73 1740.75 1756.14] 1754.96
[1768.45 1749.73 1740.75 1756.14 1754.96] 1749.28
[1749.73 1740.75 1756.14 1754.96 1749.28] 1740.24
[1740.75 1756.14 1754.96 1749.28 1740.24] 1737.81
[1756.14 1754.96 1749.28 1740.24 1737.81] 1749.74
[1754.96 1749.28 1740.24 1737.81 1749.74] 1760.87
[1749.28 1740.24 1737.81 1749.74 1760.87] 1773.77
[1740.24 1737.81 1749.74 1760.87 1773.77] 1778.25
[1737.81 1749.74 1760.87 1773.77 1778.25] 1771.8
[1749.74 1760.87 1773.77 1778.25 1771.8 ] 1770.69
[

In [13]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Price'][0],df['Price'][1],df['Price'][2],df['Price'][3],df['Price'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat*1000)

Epoch 1/200
41/41 [==============================] - 4s 59ms/step - loss: 380693.8125
Epoch 2/200
41/41 [==============================] - 2s 53ms/step - loss: 826.7518
Epoch 3/200
41/41 [==============================] - 1s 33ms/step - loss: 648.0814
Epoch 4/200
41/41 [==============================] - 1s 33ms/step - loss: 696.4119
Epoch 5/200
41/41 [==============================] - 1s 34ms/step - loss: 725.4453
Epoch 6/200
41/41 [==============================] - 1s 34ms/step - loss: 852.2914
Epoch 7/200
41/41 [==============================] - 1s 34ms/step - loss: 682.9253
Epoch 8/200
41/41 [==============================] - 1s 34ms/step - loss: 654.4001
Epoch 9/200
41/41 [==============================] - 1s 35ms/step - loss: 676.7778
Epoch 10/200
41/41 [==============================] - 1s 34ms/step - loss: 681.3618
Epoch 11/200
41/41 [==============================] - 1s 33ms/step - loss: 702.7057
Epoch 12/200
41/41 [==============================] - 1s 34ms/step - loss: 634.798

In [14]:
print(f"Prediction: {round(yhat[0][0],2)} $ Gold/USD") # onluk olarak yukarı yuvarlandı.

Prediction: 1796.2900390625 $ Gold/USD
